# Hearthstone Deck Recommendation
### Objetivo: Construir un mazo ganador para una partida de Hearthstone.

## Librerias

In [36]:
import pandas as pd
import numpy as np
import random as rd
from collections import Counter
import keras
import pickle

## Base de datos
La base de datos se obtuvo de la plataforma HearthPwn (https://www.hearthpwn.com/decks), esta contiene información sobre mazos creados por jugadores de Hearthstone. Los datos incluyen la clase del mazo, sus 30 cartas, el coste de creación, la puntuación, entre otros, pero dentro de las columnas más importantes está el rating que asignaron múltiples jugadores en la plataforma.

In [2]:
df = pd.read_csv("data/data.csv")
df.head()

,craft_cost,date,deck_archetype,deck_class,deck_format,deck_id,deck_set,deck_type,rating,title,...,card_20,card_21,card_22,card_23,card_24,card_25,card_26,card_27,card_28,card_29
0,9740,2016-02-19,Unknown,Priest,W,433004,Explorers,Tavern Brawl,1,Reno Priest,...,374,2280,2511,2555,2566,2582,2683,2736,2568,2883
1,9840,2016-02-19,Unknown,Warrior,W,433003,Explorers,Ranked Deck,1,RoosterWarrior,...,1781,1781,2021,2021,2064,2064,2078,2510,2729,2736
2,2600,2016-02-19,Unknown,Mage,W,433002,Explorers,Theorycraft,1,Annoying,...,1793,1801,1801,2037,2037,2064,2064,2078,38710,38710
3,15600,2016-02-19,Unknown,Warrior,W,433001,Explorers,NaN,0,Standart pay to win warrior,...,1657,1721,2018,2296,2262,336,2729,2729,2736,2760
4,7700,2016-02-19,Unknown,Paladin,W,432997,Explorers,Ranked Deck,1,Palamix,...,2027,2029,2029,2064,2078,374,2717,2717,2889,2889


In [3]:
df.describe()

,craft_cost,deck_id,rating,card_0,card_1,card_2,card_3,card_4,card_5,card_6,...,card_20,card_21,card_22,card_23,card_24,card_25,card_26,card_27,card_28,card_29
count,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,...,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000
mean,5645.360218,394152.425798,2.687230,166.555443,215.682814,328.068948,388.254399,465.439497,530.422084,618.649576,...,6676.817657,7879.285156,9310.508977,10780.480210,12401.890674,13709.898080,15068.109406,15955.823881,19239.277147,20537.999847
std,3927.986295,222605.617140,22.117751,515.546751,549.163776,609.109069,629.218897,710.534945,813.475129,987.250288,...,12610.593768,13750.165455,14885.066464,15862.422929,16731.209629,17295.441879,17766.351821,18039.395857,18567.676290,18686.462303
min,0.000000,18.000000,0.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
25%,2720.000000,216721.500000,1.000000,64.000000,75.000000,189.000000,237.000000,279.000000,304.000000,401.000000,...,1158.000000,1363.000000,1659.000000,1783.000000,1794.000000,1913.000000,2010.000000,2037.000000,2078.000000,2095.000000
50%,5000.000000,406046.500000,1.000000,138.000000,180.000000,285.000000,315.000000,415.000000,475.000000,559.000000,...,1940.000000,2029.000000,2061.000000,2078.000000,2275.000000,2488.000000,2577.000000,2682.000000,2901.000000,3015.000000
75%,7740.000000,590820.500000,1.000000,238.000000,285.000000,421.000000,476.000000,605.000000,643.000000,763.000000,...,2610.000000,2736.000000,2890.000000,2958.000000,38391.000000,38526.000000,38727.000000,38833.000000,38918.000000,39034.000000
max,48000.000000,749548.000000,4016.000000,41409.000000,41409.000000,41609.000000,41409.000000,41409.000000,41609.000000,41609.000000,...,41609.000000,41609.000000,41609.000000,41609.000000,41609.000000,41609.000000,41841.000000,42146.000000,42146.000000,42146.000000


In [4]:
df['deck_class'].value_counts()

deck_class
Mage       45306
Priest     44307
Paladin    42266
Warlock    38022
Druid      37891
Shaman     36457
Warrior    35944
Rogue      34794
Hunter     31245
Name: count, dtype: int64

La clase de mazo con más entradas es "Mage". Para la implementación del sistema de recomendación utilizaremos solo mazos mago.

In [5]:
df = df[df['deck_class'] == 'Mage']

## Cartas y mazos
Definimos el conjunto *cards* con las *N* cartas más repetidas en los mazos mago y *D* como el tamaño del mazo para armar.

In [7]:
cards_cols = df.iloc[:, 11:41] # columnas con las cartas en la base de datos
all_cards = (cards_cols.values.ravel()).tolist() # lista con las cartas
counter = Counter(all_cards) # contador de las cartas

N = 40#len(counter) # tamaño conjunto cartas para elegir
cards =  [item[0] for item in counter.most_common(N)] # conjunto con las N cartas más repetidas

D = 30 # tamaño mazo Hearthstone

El mazo lo definimos como un vector binario *deck* con *N* entradas y exactamente *D* iguales a 1. Este vector nos indica con la i-esima entrada si está o no (1 o 0) en el mazo la i-esima carta de *cards*. Definimos la función *binary_to_cards* que transforma el vector *deck* en una lista con la enumeración de las cartas del mazo. A continación se presenta un ejemplo de mazos.

In [8]:
def binary_to_cards(deck, cards):
    return [int(x) for x in deck*cards if x != 0]

In [9]:
deck = np.zeros(N, dtype=int) # definir mazo jugador
deck[np.random.choice(N, D, replace=False)] = 1 # generar mazo aleatorio jugador
print(f'Vector mazo: {deck}')
print(f'Cartas mazo: {binary_to_cards(deck, cards)}')

Vector mazo: [1 1 1 0 0 1 0 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1
 1 1 1]
Cartas mazo: [662, 315, 555, 614, 825, 564, 1783, 2275, 192, 587, 195, 172, 2874, 621, 138, 1080, 1659, 1801, 1927, 1793, 2262, 251, 113, 2875, 1928, 430, 2037, 1087, 2050, 2078]


## Transiciones y acciones
El proceso para mejorar el mazo *deck* consta de aplicar múltiples veces la función de transición *transition* determinada por una acción. Una acción *action* es una tupla con las cartas del reemplazo que modifica exactamente una carta del mazo por una carta que actualmente no está incluida. A continuación, un ejemplo del uso de la función *transition*, en donde se elige de forma aleatoria la acción.

In [10]:
def transition(deck, action):
    deck[action[0]] = 0 # eliminamos la carta antigua del mazo
    deck[action[1]] = 1 # agregamos la carta nueva al mazo
    return deck

In [11]:
print(f'Mazo antiguo: {deck}')
print(f'Cartas mazo antiguo: {binary_to_cards(deck, cards)}')

old_card = np.random.choice(np.where(deck == 1)[0]) # elegimos aleatoriamente una carta que está en el mazo
new_card = np.random.choice(np.where(deck == 0)[0]) # elegimos aleatoriamente una carta que no está en el mazo
action = (old_card, new_card) # definimos la acción

print()
print(f'Acción: ({action[0]},{action[1]})')
print()

new_deck = transition(deck, action)
print(f'Mazo nuevo: {new_deck}')
print(f'Cartas mazo nuevo: {binary_to_cards(new_deck, cards)}')

Mazo antiguo: [1 1 1 0 0 1 0 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1
 1 1 1]
Cartas mazo antiguo: [662, 315, 555, 614, 825, 564, 1783, 2275, 192, 587, 195, 172, 2874, 621, 138, 1080, 1659, 1801, 1927, 1793, 2262, 251, 113, 2875, 1928, 430, 2037, 1087, 2050, 2078]

Acción: (36,24)

Mazo nuevo: [1 1 1 0 0 1 0 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0
 1 1 1]
Cartas mazo nuevo: [662, 315, 555, 614, 825, 564, 1783, 2275, 192, 587, 195, 172, 2874, 621, 138, 1080, 1659, 2064, 1801, 1927, 1793, 2262, 251, 113, 2875, 1928, 430, 1087, 2050, 2078]


## Win Rate
Para elegir las acciones que nos lleven a encontrar un mazo ganador usaremos el *win_rate* de este mazo. Entonces, para elegir la próxima acción buscaremos cual es la que maximiza el *win_rate* del mazo.

Para hacer una función predictora del *win_rate* entrenamos distintos modelos de machine learning, esto se realizó en el notebook *win_rate_prediction.ipynb*. A continuación, se importan estos modelos ya entrenados.

In [12]:
model = keras.models.load_model("win_rate_NN.keras") # importación modelo red neuronal
model.summary()

with open("encoder", "rb") as f:
    encoder = pickle.load(f)
with open("win_rate_RF.pkl", "rb") as f:
    rf = pickle.load(f) # importación modelo random forest

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │         1,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_dropout                │ (None, 32)             │             0 │
│ (GaussianDropout)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │         1,650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_dropout_1              │ (None, 50)             │             0 │
│ (GaussianDropout)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,883 (54.23 KB)

 Trainable params: 4,627 (18.07 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 9,256 (36.16 KB)

Ahora, definimos la función *win_rate* que recibe el mazo en el formato enumerado de cartas y dependiendo del modelo elegido entrega el valor del win rate.

In [34]:
def win_rate(deck, encoder = encoder, model = model):
  print(deck)
  mazo_gen_num = pd.DataFrame([{f"card_{i}": (deck[i] if i>-1 else 5645) for i in range(-1,30)}]).rename(columns={"card_-1":"craft_cost"})
  hero = "Mage"
  mazo_gen_cat = pd.DataFrame([{"hero":hero}])
  encoded_categorical = encoder.transform(mazo_gen_cat)
  # # Step 3: Convert the encoded data back to a DataFrame
  encoded_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out())
  # # Step 4: Concatenate the numerical column with the one-hot encoded categorical columns
  final_df = pd.concat([mazo_gen_num, encoded_df], axis=1).rename(columns={"craft_cost":"dust"})
  results = model.predict(final_df)
  if isinstance(results[0],np.ndarray):
    return results[0][0]
  else:
    return results[0]

Finalmente, imprimimos el win rate aproximado para el mazo *deck_p* usando los dos modelos. Primero en base a la red neuronal entrenada y después mediante un random forest.

In [25]:
print('neural network:', win_rate(binary_to_cards(deck, cards))) # red neuronal
print('random forest:', win_rate(binary_to_cards(deck, cards),model=rf)) # random forest

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
neural network: 51.131912
random forest: 50.37666666666665


## Implementación 

Con la función de *win_rate* ya definida podemos decidir cual es la siguiente acción más conveniente a realizar. Vamos a comenzar con un mazo *deck* y realizar acciones hasta que ya no sea conveniente seguir modificando el mazo, estas serán a los más *D*, ya que a lo más cambiamos todo el mazo para encontrar el óptimo. A continuación, definimos la función *next_action* que a partir del mazo actual encuentra la siguiente acción más conveniente a realizar según el *win_rate*, incluyendo la acción de preservar el mazo igual.

In [32]:
def next_action(deck, cards):
    current_win_rate = win_rate(binary_to_cards(deck, cards))
    best_win_rate = current_win_rate
    best_action = None
    
    zeros = [index for index, value in enumerate(deck) if value == 0]
    ones = [index for index, value in enumerate(deck) if value == 1]
     
    for zero in zeros:
        for one in ones: 
            temp_deck = deck.copy()  
            action = (one, zero)
            temp_deck = transition(temp_deck, action)
            temp_win_rate = win_rate(binary_to_cards(temp_deck, cards))
            if temp_win_rate > best_win_rate:
                best_win_rate = temp_win_rate
                best_action = action
    
    return best_action    

Ahora con la función *deck_reck* podemos hacer efectiva la mejor acción sucesivamente hasta llegar a *D* acciones o haber generado un mazo óptimo.

In [15]:
def deck_rec(deck, cards):
    for i in range(D):
        action = next_action(deck, cards)
        if action == None:
            break
        else:
            deck = transition(deck, action)
    return deck

In [16]:
print(f'Mazo inicial: {binary_to_cards(deck, cards)}')
print(f'Win rate inicial: {win_rate(binary_to_cards(deck, cards))}')
new_deck = deck_rec(deck, cards)
print(f'Mazo final: {binary_to_cards(new_deck, cards)}')
print(f'Win rate final: {win_rate(binary_to_cards(new_deck, cards))}')

Mazo inicial: [662, 315, 555, 614, 825, 564, 1783, 2275, 192, 587, 195, 172, 2874, 621, 138, 1080, 1659, 2064, 1801, 1927, 1793, 2262, 251, 113, 2875, 1928, 430, 1087, 2050, 2078]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Win rate inicial: 54.76186752319336
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━

## Comparación de resultados
Finalmente vamos a comparar esta implementación con los siguientes métodos.

### Random
Entrega de forma aleatoria un mazo de cartas de la base de datos.

In [17]:
def random_deck():
  row = df.sample(n=1)
  id = row['deck_id'].tolist()[0]
  cards_cols = row.iloc[:, 11:41]
  cards_list = (cards_cols.values.ravel()).tolist() 
  return id, cards_list

### Most popular
Retorna el mazo con el mejor rating de la base de datos.

In [18]:
def most_popular_deck():
  row = df[df['rating'] == df['rating'].max()]
  cards_cols = row.iloc[:, 11:41]
  cards_list = (cards_cols.values.ravel()).tolist() 
  return cards_list

### Best Similar 
Recibe un mazo y retorna un mazo similar de la base de datos con mejor rating. Para definir la similitud, se implementó una función *difference* que calcula la cantidad de cartas diferentes entre los mazos. De esta forma, el modelo también recibe como parámetro un *delta* que define la máxima posible diferencia entre el mazo retornado y el original.

In [20]:
def difference(id_1, id_2):
  deck_1 = df[df['deck_id'] == id_1].iloc[:, 11:41].values.flatten()
  deck_2 = df[df['deck_id'] == id_2].iloc[:, 11:41].values.flatten()
  common = len(np.intersect1d(deck_1, deck_2))
  diff = len(deck_1) - common
  return diff

def best_similar_deck(my_deck_id, delta):
  best_decks = df[df['rating'] > 2]
  best_deck_id = my_deck_id
  for index, row in best_decks.iterrows():
    new_deck_id = row['deck_id']
    new_deck_rating = row['rating']
    if difference(my_deck_id, new_deck_id) <= delta:
      best_deck_rating = df.loc[df['deck_id'] == best_deck_id, 'rating'].values[0]
      if new_deck_rating > best_deck_rating:
        best_deck_id = new_deck_id

  row = df[df['deck_id'] == best_deck_id]
  cards_cols = row.iloc[:, 11:41]
  cards_list = (cards_cols.values.ravel()).tolist() 
  return cards_list

Entonces, a continuación vamos a seleccionar de forma aleatoria un mazo de la base de datos y vamos a comparar los distintos sistemas recomendadores usando el *win_rate*.

In [21]:
original_id, original_deck = random_deck() # generamos mazo aleatorio
print(f'Mazo inicial: {original_deck}')
print(f'Win rate inicial: {win_rate(original_deck)}')

Mazo inicial: [113, 113, 195, 195, 315, 315, 405, 405, 555, 555, 662, 662, 748, 748, 1737, 1737, 2275, 2275, 38418, 39169, 39169, 39715, 39715, 39767, 39767, 40299, 40299, 40409, 40583, 40583]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Win rate inicial: 53.06285858154297


In [22]:
id_random, deck_random = random_deck() # generamos recomendación aleatoria
print(f'Recomendación random: {deck_random}')
print(f'Win rate random: {win_rate(deck_random)}')

Recomendación random: [138, 192, 315, 315, 405, 405, 430, 457, 555, 555, 564, 614, 614, 662, 662, 749, 1004, 1004, 1080, 1084, 1084, 2572, 2958, 2958, 38418, 38505, 39169, 39426, 39715, 39841]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Win rate random: 53.24312210083008


In [23]:
deck_most_popular = most_popular_deck() # generamos recomendación más popular
print(f'Recomendación más popular: {deck_most_popular}')
print(f'Win rate más popular: {win_rate(deck_most_popular)}')

Recomendación más popular: [77, 315, 315, 405, 405, 555, 564, 564, 614, 635, 662, 662, 825, 825, 1004, 38547, 38547, 38725, 38725, 38857, 38859, 38859, 38863, 38863, 38868, 38868, 38900, 38900, 39715, 39715]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Win rate más popular: 53.383628845214844


In [24]:
deck_best_similar = best_similar_deck(original_id, 25) # generamos recomendación mejor similar
print(f'Recomendación mejor similar: {deck_best_similar}')
print(f'Win rate mejor similar: {win_rate(deck_best_similar)}')

Recomendación mejor similar: [77, 315, 315, 405, 405, 555, 564, 564, 614, 635, 662, 662, 825, 825, 1004, 38547, 38547, 38725, 38725, 38857, 38859, 38859, 38863, 38863, 38868, 38868, 38900, 38900, 39715, 39715]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Win rate mejor similar: 53.383628845214844


In [40]:
deck = np.zeros(N, dtype=int)
for i in range(N):
    for card in original_deck:
        if card == cards[i]:
            deck[i] = 1
zeros = [index for index, value in enumerate(deck) if value == 0]
ones = [index for index, value in enumerate(deck) if value == 1]
new_ones = rd.sample(zeros,D-len(ones))
for i in new_ones:
    deck[i] = 1
cards_subset = cards[:N]
print(deck)
deck_our_rec = deck_rec(deck,cards_subset)
print(f'Nuestra recomendación : {binary_to_cards(deck_our_rec, cards_subset)}')
print(f'Win rate nuestra recomendación: {win_rate(binary_to_cards(deck_our_rec, cards_subset))}')

[1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1
 1 1 1]
[662, 315, 555, 1004, 405, 77, 564, 1783, 395, 2275, 1084, 192, 587, 195, 621, 138, 1080, 1659, 2064, 1927, 1793, 2572, 2262, 113, 2875, 1928, 2037, 1087, 2050, 2078]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[315, 555, 1004, 405, 614, 77, 564, 1783, 395, 2275, 1084, 192, 587, 195, 621, 138, 1080, 1659, 2064, 1927, 1793, 2572, 2262, 113, 2875, 1928, 2037, 1087, 2050, 2078]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[662, 555, 1004, 405, 614, 77, 564, 1783, 395, 2275, 1084, 192, 587, 195, 621, 138, 1080, 1659, 2064, 1927, 1793, 2572, 2262, 113, 2875, 1928, 2037, 1087, 2050, 2078]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[662, 315, 1004, 405, 614, 77, 564, 1783, 395, 2275, 1084, 192, 587, 195, 621, 138, 1080, 1659, 2064, 1927, 1793, 2572, 2262, 113, 2875, 1928, 2037, 1087, 2050, 2078]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[662, 315, 555, 405, 614, 77, 564, 1783, 395, 2275, 1084, 192, 587, 195, 621, 138, 1080, 1659, 20